In [38]:
# Import library
import pickle
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

In [39]:
# Load Saved Model
with open('fix_model_facenet_160x160.pkl', 'rb') as f:
    fix_model_facenet = pickle.load(f)

In [40]:
# Initiate MTCNN Model
img_detector = MTCNN()

In [41]:
# Load the image
t_img = cv.imread("data/abed/abed_2.jpg")

# Convert the image to RGB
t_img = cv.cvtColor(t_img, cv.COLOR_BGR2RGB)

# Resize the image to the new size while maintaining aspect ratio
new_size = (480, 480)
height, width, _ = t_img.shape
scale = min(new_size[0] / width, new_size[1] / height)
new_width = int(width * scale)
new_height = int(height * scale)
resized_img = cv.resize(t_img, (new_width, new_height))


In [42]:
# Detect faces in the image
detections = img_detector.detect_faces(resized_img)
if not detections:
    raise Exception("No faces detected in the image.")

1/1 [==============================] - 0s 129ms/step


In [43]:
# Extract the coordinates and size of the bounding box from the first detection result
x, y, w, h = detections[0]['box']

In [44]:
# Instantiate the FaceNet model
embedder = FaceNet()

In [45]:
# Define a function to get the embedding of a face image
def get_embedding(face_img):
    face_img = face_img.astype('float32')  # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0)  # 4D (Nonex160x160x3)
    yhat = embedder.embeddings(face_img)
    return yhat[0]  # 512D image (1x1x512)

In [46]:
# Crop the detected face region from t_img using the bounding box coordinates
face_img = resized_img[y:y+h, x:x+w]

# Resize the cropped face image to 160x160
face_img = cv.resize(face_img, (160, 160))

# Get the embedding (feature vector) for the resized face image using the get_embedding function (Facenet)
test_embedding = get_embedding(face_img)

1/1 [==============================] - 2s 2s/step


In [47]:
# Convert the embedding to the correct shape (2D array)
test_embedding = np.expand_dims(test_embedding, axis=0)

# Make Prediction
predict = fix_model_facenet.predict(test_embedding)
predict_proba = fix_model_facenet.predict_proba(test_embedding)

In [48]:
# Decode the prediction
encoder = LabelEncoder()
LABEL = ['abed', 'budi', 'gibran', 'iyal', 'vicky']
encoder.fit(LABEL)

# Output the predicted label and confidence score
predicted_label = encoder.inverse_transform(predict)[0]
confidence_score = predict_proba[0][predict[0]]

# Check confidence score and determine if the prediction should be considered unknown
if confidence_score < 0.5:
    predicted_label = "unknown"

# Print predicted label and confidence score
print(f"Predicted Label: {predicted_label}")
print(f"Confidence Score: {confidence_score:.2f}")

Predicted Label: abed
Confidence Score: 0.88


In [49]:
# List all probabilities
print("Probabilities for each class:")
for label, proba in zip(LABEL, predict_proba[0]):
    print(f"{label}: {proba:.2f}")

Probabilities for each class:
abed: 0.88
budi: 0.01
gibran: 0.05
iyal: 0.04
vicky: 0.01
